# Easing into BMB coding in your research or classroom
## A Discover BMB 2024 Workshop by Paul Craig and Wally Novak


![Paul Craig and Wally Novak](images/paul_wally.png)
***


In this portion workshop we will work with protein sequences. You will learn to:

1. Search for a sequence directly
2. Perform a BLAST search using your own sequence and control the number of hits returned
3. Perform some simple analyses on your returned data
4. Install and use additional sequence software (if time!) 
5. Reduce sequence redundancy and keep sequence diversity (if time!) 
6. Produce a sequence alignment and look for conserved residues (if time!) 


***
## 1. Importing a known sequence directly from Entrez

Entrez is a molecular biology database system that provides integrated access to nucleotide and protein sequence data, gene-centered and genomic mapping information, 3D structure data, PubMed MEDLINE, and more. The system is produced by the National Center for Biotechnology Information (NCBI) and is available via the Internet at https://www.ncbi.nlm.nih.gov/Web/Search/entrezfs.html

### Retrieving sequence IDs using a search term.

Many of us may have used the Entrez website to search for a sequence. This will do much the same in the Juptyer Notebook environment. There are a couple things that are important to note. First, we will set two text variables. To set a text variable, we will use quotes around the text we are entering.

<span style='color:Blue'>*Setting a text variable example: x = "Jupyter is fun."*</span>

This simply sets the variable, x to Jupter is fun. Generally, it is good coding practice to use variable names that are informative.

#### In the code below, please edit the variable Entrez.email and the variable search_term. 

<span style='color:green'>**Please enter your own email address in quotes for email**</span>. Note that for search term we will use the + symbol to indicate 'and'. Please assign the variable search_term to <span style='color:green'>**"creatine+kinase+muscle+[homo sapiens (humans)]"**</span>

The code cell below is heavily commented so it is easier to see what each line is doing.

#### Run the cell.

<img src="images/run_button.png" width="100" align="left" />Then run the cell. **To run the cell, make sure it is selected (a blue bar appears to at the left) the either click the  Run button or, using the keyboard, hold "Shift+Return."** I won't explicitly tell you to run cells anymore, so read carefully and run them as they appear. Hoever, you may have to do some editing of the code cell first!

In [ ]:
# this imports the necessary tools to search Entrez
from Bio import Entrez   

# always tell the NCBI who you are!
Entrez.email = "your_email@school.edu" 

# this sets a variable for us to use as the search
search_term = "terms to search"

# this searches Entrez and places the results in the variable ID_query
# other database (db) terms could include nucleotide, pubmed, or gene
# for other databases see https://www.ncbi.nlm.nih.gov/books/NBK25497/
ID_query = Entrez.esearch(db='protein', term=search_term)

# Reading the query
ID_records = Entrez.read(ID_query)

# This closes the Entrez query    
ID_query.close()

Note that we don't see any output for the above commands. We need to see what is stored in ID_query. The for loop below iterates through each record in ID_records and stores it in a variable called seq_id. The next line prints what is in seq_id.

In the coding cell below, type or paste:

```python
# this for loop iterates through the IDs returned
for seq_id in ID_records['IdList']:
    print(seq_id)
```
#### Then run the cell.


***
### Retrieving information about the sequence IDs. 
Now that we have some sequence IDs related top our search, let's download the sequences themselves so we can choose one for analysis.

For more information on the fasta format for protein sequences, check out this source: https://www.ncbi.nlm.nih.gov/WebSub/html/help/protein.html

The first bit of code: **```SEQ_LIST = []```** creates an empty list. 

Next is anther for loop that iterates through the records much like we did in the above cell.

#### Then run the cell.

In [ ]:
SEQ_LIST = []

# This for loop takes each sequence ID and returns its sequence in fasta format
for seq_id in ID_records['IdList']:
    print("Retrieving fasta sequence for ID: %s" % seq_id)
    seq_query = Entrez.efetch(db ='sequences', id=seq_id, rettype='fasta', retmode='text')
    SEQ_LIST.append(seq_query.read())
    
# This closes the sequence query
seq_query.close()

Since we have returned only a few sequences, we can examine these by eye. But we need to print out the sequences. Let's see if you can write a for loop that iterates through SEQ_LIST and prints the sequence. You can enter the code in the code box below and **run it**, but if you get stuck see the hint just underneath this box.

<details>
    <summary><b>Click here to see some sample code to iterate through our sequence list.</b></summary>

Here is the simplest code you could use:
```python
for seq in SEQ_LIST:
    print(seq)
```

This code adds a little je ne sais quoi. It creates a counter that shows us what number each sequence is in the list. Note that python coding starts at zero!
    
```python
i = 0 # this is a simple variable to count up in the list
for seq in SEQ_LIST:
    print("This is the sequence stored in SEQ_LIST[%i]" % i)
    print(seq)
    i = i + 1
```
    
</details>

Select a sequence in the output above to move forward with from the list. Python lists start couting at zero. If you used the fancy code, the output above includes which item in the list corresponds to each sequence. If not, you can count through the sequences, starting with zero! Edit the code below to set the variable seq to the SEQ_LIST selection of your choice. For example, the first sequence would be ```SEQ_LIST[0]```. (Really only one of the above looks like a poorer choice than the others. :)

In [ ]:
seq = SEQ_LIST[0]

print(seq)

***
## 2. Performing a BLAST search

Now that we have a protein sequence, we can ask, What proteins are similar to the one I have selected? We can do this using the Basic Local Alignment Tool (BLAST) from the NCBI. For more information on BLAST check out this page: https://blast.ncbi.nlm.nih.gov/Blast.cgi



To perform a BLAST search we will first need to import the right tools. In the code cell below copy or type:

```python
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
```

#### Then run the cell.

### Performing the BLAST query

The next coding cell defines the variable query using blastp (to look for proteins), against the refseq-protein database, using our sequence (defined in seq). Further for the purpose of this demonstration we will have BLAST return 500 sequences. In practice, this number might be something larger, like 10000; however, this will take a little longer to return results. Finally, we have asked the BLAST query to return 0 alignments and to use the XML format.

In [ ]:
# defines the query as a blastp search against the nr database. 
#Other database options include: refseq_protein, pdb, and swissprot
query = NCBIWWW.qblast('blastp','refseq_protein', seq, hitlist_size=500, alignments=0, format_type = 'XML')

# Open the file you will 'write' (w) into
results = open('data/CK_blastp.xml', 'w')
print("Results are stored in the file data/CK_blastp.xml")

# Write the file with the query results
results.write(query.read())

# Close the file
results.close()

# Close the query
query.close()

We see that we have the results of the blast search stored in the qblast_blastp.xml. XML (extensible markup language) makes the file simple to process, but difficult to read.

### Parsing the BLAST output and writing a fasta file

We will use the NCBIXML.parse command to process the XML file. We will store the results in a traditional fasta sequence file that we saw above.

Let's walk through the code below.

In [ ]:
#This sets our filename to store our fasta sequence data in, and opens the file for writing.
outfile="data/ck_seqs.fasta"
f = open(outfile, 'w')

# Sets the variable results_handle as the opened xml file
result_handle = open('data/CK_blastp.xml')

#This parses the xml file into blast_records, which is iterable
blast_records = NCBIXML.parse(result_handle)

#This for loop iterates through all the records in blast_records.
#It then looks at the alignments in the record, which stores the HSP (high scoring pair)
for record in blast_records:
    for alignment in record.alignments:
        for hsp in alignment.hsps:
            #now we get the ID for the hit (this splits the line on '>' only taking the first [0th] item)
            hit_id = alignment.title.split('>')[0]
            #write out the hit ID to the file, followed by a carriage return
            f.write(">"+hit_id+'\n')
            #write out the subject of the search, which is the hit, with no gaps, followed by two carriage returns
            f.write(hsp.sbjct.replace('-', '')+'\n\n')

#This closes the results handle and the output file to which we were writing.
result_handle.close()
f.close()

At this stage you should know how to retrieve a sequence from Entrez and perform a BLAST search while controlling the number of hits returned. Now let's look at some "simple" ways of analyzing the sequence data.

***
## 3. Simple analyses of BLAST data using biopython and matplotlib

Biopython (https://biopython.org/) is a set of tools for manipulating sequence data in python. In fact, many of the tools we used above are part of biopython. Paul Craig introduced you to matplotlib in the first half of this workshop. We will extract some information about the lengths of the sequences in our blast results and create some simple plots to visualize the data.

### Parse the fasta file into a list of records

First, we will import the SeqIO (sequence input and output) tool from biopython and parse the fasta file into a list called records.

In [ ]:
from Bio import SeqIO # imports the SeqIO function from Biopython

records = list(SeqIO.parse("data/ck_seqs.fasta", "fasta"))     # reads the fasta file into a list of records 
print("Finished storing the FASTA file in the list called \"records\".")

Now that we have our sequences stored in a list, we can ask how many records there are in the entire list.

In the code cell below, type, then run:
```python
print("There are %i sequences in your file.\n" % len(records))  
```

Let's look at what is stored for each record.

In the code cell below, type, then run:

```python
print(records[0])
```

This gives us an abbreviated view of the information present in this list.

We can also print sequence or ID information about each record. Let's look at the last item in the list, which we can access with the list position of -1.

In the code cell below, type, then run:

```python
print(records[-1].description) 
print(records[-1].seq)
```


Okay, time to incorporate our knowledge of for loops to return the first 10 record descriptions.

```python
print("The first 10 sequence record ids are:\n")
#This loop counts from 0 to 9
for i in range(10):                                        
    print(records[i].description)                                        
```


### Plotting information about sequences using matplotlib

It is often easier to view some details about datasets by plotting them. We can use matplotlib to plot information about the lengths of the sequences returned in our blast search. 

Let's walk through the code below that harvests the sequence lengths from our fasta file.

In [ ]:
#Create an empty list to hold the sequence lengths for each record
lengths = []

#Loop through the records
for record in records:
    #Use the len command to determine the length of the record sequence and add it to the list called lengths
    lengths.append(len(record.seq))

#Sort the lengths of the sequences
lengths.sort()

This code stored the lengths of the sequences in a sorted list. Let's look at a few using a for loop!

Try to write your own python code to iterate through the first 10 items in list **lengths** and print them. You can open up the code below if you want some help.


<details>
    <summary><b>Click here to see some sample code to iterate through our length list.</b></summary>

Here is some simple code you could use:
```python
for i in range(10):
    print(lengths[i])
```
    
</details>

Next let's plot the data using matplotlib to visualize the lengths returned in our blast search. Let's copy and paste this code into the code cell below and run it.

```python
#First import matplotlib
import matplotlib.pyplot as plt

#Here we make the plot.
#the first line says we will make a histogram using lengths. It will bin the sequences into 100 bins.
plt.hist(lengths, bins = 100)
#The next two lines set the axes labels
plt.xlabel('seq length')
plt.ylabel('count')
#Then draw the plot
plt.show()
```

We can see from the data that we have a large number of sequences that are 375-385 amino acids in length. Let's try to create our own plot using the above cell as a guide. **This time let's create a plot that only looks at the 375-385 sequence length region.**

We can use the same code above, but let's add one command before the plt.show() line: **```plt.xlim(375,385)```**

<details>
    <summary><b>Click here to see the code to create a new plot.</b></summary>

Here is some simple code you could use:
```python
plt.hist(lengths, bins = 100)
#The next two lines set the axes labels
plt.xlabel('seq length')
plt.ylabel('count')
#The next line sets the limits of the x-axis!
plt.xlim(375,385)
plt.show()
```
    
</details>

Most of the sequences fall between 375 and 385 amino acids in length. Let's remove shorter and longer sequences from our analysis.

Let's walk through the code below and then run it.

In [ ]:
#We set the variable trimmed_file to a new output file in the data folder.
trimmed_file = "data/ck_blast_trim.fasta"

#We set the variables for short and long lengths here.
small_len = 375
large_len = 385

#Open the output file for writing
trimmed = open(trimmed_file, 'w')

#A simple for loop that checks the length of each sequence, 
#only writing the seq to the new file if it is in the range indicated
for record in records:
    if len(record.seq) > small_len and len(record.seq) < large_len: 
        SeqIO.write(record, trimmed, 'fasta')

#The next lines parse the newly trimmed file and tell us how many sequences we now have.
records2 = list(SeqIO.parse("data/ck_blast_trim.fasta", "fasta"))
print("We now have %i sequences in our fasta file." % len(records2))

## 4. Installing software

The Jupyter Notebook is outstanding for running your own python code. However, many tools were meant to run in a different environment (unix!). We will use two of these tools - Clustal Omega (for sequence alignments) and CD-HIT for reducing redundancy in sequence datasets.

To run a shell command outside of the Jupyter Notebook you simply type a '!' (also called "bang") before the command. Independently installing software in a unix shell can be quite tricky. There are often lots of other related pieces of software than must be installed. These other software items are called dependencies. To alleviate this issue, we will use the software package Mamba, which will install the sofware we want and any dependencies needed. First let's install clustalomega.

In the coding cell below, type or paste: 

```python
!mamba install --yes bioconda::clustalo
```

This command tells mamba to install clustalo from the package manager, conda. The '--yes' flag tells the installer to answer yes and continue for any questions that pop up during the installation process. You can find out more about available bioconda packages here: https://bioconda.github.io/index.html

Run the cell.


#### N.B. Please make sure to wait for the code cell to finish running before moving on the next cell. 
You will know it is still running if you see an asterisk like this: <span style='color:blue'>In [*]:</span>
If a cell has not been run it is empty, e.g. <span style='color:blue'>In [ ]:</span>, and if it has completed you will see a number in it, e.g. <span style='color:blue'>In [1]:</span>

Next install cd-hit. You can find out more about cd-hit here: https://sites.google.com/view/cd-hit

In the coding cell below, type or paste: 

```python
!mamba install --yes bioconda::cd-hit
```

Run the cell.


We now have access to ClustalOmega (a sequence alignment tool) and CD-Hit (used to reduce the size of sequence lists). It is easy to see how we can run each program using the --help flag.

First, type and then run:

```python
!clustalo --help
```

The above output shows you all the flags you can pass ClustalOmega when running it.

Next, look at the help for CD-Hit.

Type and then run:

```python
!cd-hit --help
```


## 5. Reducing the sequence set with CD-Hit
Okay, now let's use some of this software to reduce our sequence dataset and then align our sequences.

In the code cell below, type and run:

```python
!cd-hit -i data/ck_blast_trim.fasta -o data/ck_90.fasta -c 0.90 -n 5
```

This takes the ck_blast_trim.fasta file as input, and outputs a file with sequences that are no more than 90% identical.


## 6. Aligning Sequences with ClustalOmega

The output above tells us how many clusters (sequences), we have in our new output file. 

Next, we will align the sequences using clustalo.

In the code cell below, type and run:

```python
!clustalo -v -i data/ck_90.fasta -o data/ck_90.aln --outfmt=fasta --force
```
This takes the output from CD-Hit and alignes those sequences in fasta format.

## 7. Viewing a multiple sequence alignment

Finally, we will install 2 more packages, panel and bokeh to build a multiple sequence alignment (MSA) viewer. You can find out more information about each here:

https://panel.holoviz.org/
https://bokeh.org/

Type then run:

```python
!mamba install --yes panel bokeh
```


The MSA code below is from: **https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner**

and the adaptation to protein alignment is from: **Nichole Orench-Rivera**

Run the cell below to import the necessary tools.

In [ ]:
import os, io, random
import string
import numpy as np

from Bio.Seq import Seq
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

import panel as pn
import panel.widgets as pnw
pn.extension()

from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Plot, Grid, Range1d
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot

You are welcome to look at the code below, but it is complicated and unecessary for viewing a sequence alignment. To see more of the code you can click the two-headed arrows at right or the arrowheads at left.

Go ahead and run the code below.

In [ ]:

def get_colors(seqs):
    #make colors for bases in sequence
    text = [i for s in list(seqs) for i in s]
    #Use Zappo color scheme
    clrs =  {'K':'red',
             'R':'red',
             'H':'red',
             'D':'green',
             'E':'green',
             'Q':'blue',
             'N':'blue',
             'S':'blue',
             'T':'blue',
             'A':'blue',
             'I':'blue',
             'L':'blue',
             'M':'blue',
             'V':'blue',
             'F':'orange',
             'Y':'orange',
             'W':'orange',
             'C':'blue',
             'P':'yellow',
             'G':'orange',
             '-':'white'}
    colors = [clrs[i] for i in text]
    return colors
    
def view_alignment(aln, fontsize="8pt", font="courier", plot_width=900):
    """Bokeh sequence alignment view"""

    #make sequence and id lists from the aln object
    seqs = [rec.seq for rec in (aln)]
    ids = [rec.id for rec in aln]    
    text = [i for s in list(seqs) for i in s]
    colors = get_colors(seqs)    
    N = len(seqs[0])
    S = len(seqs)    
    width = .4

    x = np.arange(1,N+1)
    y = np.arange(0,S,1)
    #creates a 2D grid of coords from the 1D arrays
    xx, yy = np.meshgrid(x, y)
    #flattens the arrays
    gx = xx.ravel()
    gy = yy.flatten()
    #use recty for rect coords with an offset
    recty = gy+.5
    h= 1/S
    #now we can create the ColumnDataSource with all the arrays
    source = ColumnDataSource(dict(x=gx, y=gy, recty=recty, text=text, colors=colors))
    plot_height = len(seqs)*15+50
    x_range = Range1d(0,N+1, bounds='auto')
    if N>80:
        viewlen=80
    else:
        viewlen=N
    #view_range is for the close up view
    view_range = (0,viewlen)
    tools="xpan, reset, save" #xwheel_zoom,

    #entire sequence view (no text, with zoom)
    p = figure(title=None, width=plot_width, height=50,
               x_range=x_range, y_range=(0,S), tools=tools,
               min_border=0, toolbar_location='below')
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                 line_color=None, fill_alpha=0.6)
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.grid.visible = False



    #sequence text view with ability to scroll along x axis
    p1 = figure(title=None, width=plot_width, height=plot_height,
                x_range=view_range, y_range=ids, tools="xpan,reset",
                min_border=0, toolbar_location='below')#, lod_factor=1)          
    glyph = Text(x="x", y="y", text="text", text_align='center',text_color="black", 
                 text_font="helvetica",text_font_size=fontsize)
    rects = Rect(x="x", y="recty",  width=1, height=1, fill_color="colors",
                line_color=None, fill_alpha=0.4)
    p1.add_glyph(source, glyph)
    p1.add_glyph(source, rects)

    p1.grid.visible = False
    p1.xaxis.major_label_text_font_style = "bold"
    p1.yaxis.minor_tick_line_width = 0
    p1.yaxis.major_tick_line_width = 0

    p = gridplot([[p],[p1]], toolbar_location='below')
    #p = gridplot([[p]], toolbar_location='below')
    return p


Here is the last bit of magic! The code below sets the alignment to the ck_90.aln fasta alignment we produced above. The other two lines call the code to make a beautiful and interactive MSA viewer!

Type or copy the code below in the code cell and run it!

```python
aln = AlignIO.read('data/ck_90.aln','fasta')
p = view_alignment(aln)
pn.pane.Bokeh(p)
```

I hope you have learned a little about performing sequence analyses in a python-based Jupyter notebook. If you have follow up questions or a desire to collaborate on producing exercises for biochemistry classes or labs, don't hesitate to reach out!